In [75]:
from scipy.stats import pearsonr
import os
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WhitespaceTokenizer
import nltk, string
import re
import gensim
from gensim import corpora
from nltk.corpus import stopwords
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics.pairwise import cosine_similarity as cs
from sklearn.metrics.pairwise import manhattan_distances as md
from sklearn.metrics.pairwise import euclidean_distances as ed
from sklearn.metrics import jaccard_similarity_score as jsc
from sklearn.neighbors import DistanceMetric
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import numpy as np
from nltk.metrics import jaccard_distance
from sklearn.neural_network import MLPRegressor
import csv
import warnings
from autocorrect import spell
warnings.filterwarnings('ignore')

## Statement
- Use data set and description of task Semantic Textual Similarity in SemEval 2012.
- Implement some approaches to detect paraphrase using sentence similarity metrics.
    + Explore some lexical dimensions. (Only word)
    + Explore the syntactic dimension alone. (Word respect to sentence)
    + Explore the combination of both previous.
- Add new components at your choice (optional).
- Compare and comment the results achieved by these approaches among them and among the official results.
- Send files to raco in IHLT STS Project before the oral presentation:
    + Jupyter notebook: sts-[Student1]-[Student2].ipynb
    + Slides: sts-[Student1]-[Student2].pdf


In [62]:
train_path = 'data/train/'
test_path = 'data/test-gold/'

def load_and_concat(data_path):
    files = os.listdir(data_path)
    all_data = pd.DataFrame(columns=['sentence0','sentence1'])
    all_labels = pd.DataFrame(columns=['labels'])
    for file in files: 
        path = data_path + file
        if 'input' in file:
            print(path)
            fd = pd.read_csv(path, sep='\t', lineterminator='\n', names=['sentence0','sentence1'], header=None, quoting=csv.QUOTE_NONE)
            all_data = all_data.append(fd)
            fd = pd.read_csv(path.replace('input','gs'), sep='\t', lineterminator='\n', names=['labels'], header=None, quoting=csv.QUOTE_NONE)
            all_labels = all_labels.append(fd)
    return all_data, all_labels 

train_df, train_gs = load_and_concat(train_path)
test_df, test_gs = load_and_concat(test_path)

train_df.shape, train_gs.shape,test_df.shape, test_gs.shape

data/train/STS.input.MSRpar.txt
data/train/STS.input.MSRvid.txt
data/train/STS.input.SMTeuroparl.txt
data/test-gold/STS.input.MSRpar.txt
data/test-gold/STS.input.MSRvid.txt
data/test-gold/STS.input.SMTeuroparl.txt
data/test-gold/STS.input.surprise.SMTnews.txt
data/test-gold/STS.input.surprise.OnWN.txt


((2234, 2), (2234, 1), (3108, 2), (3108, 1))

In [27]:
train_df.head()

,sentence0,sentence1
0,But other sources close to the sale said Viven...,But other sources close to the sale said Viven...
1,Micron has declared its first quarterly profit...,Micron's numbers also marked the first quarter...
2,The fines are part of failed Republican effort...,"Perry said he backs the Senate's efforts, incl..."
3,"The American Anglican Council, which represent...","The American Anglican Council, which represent..."
4,The tech-loaded Nasdaq composite rose 20.96 po...,The technology-laced Nasdaq Composite Index <....


In [63]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    w_tokenizer = WhitespaceTokenizer()
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

def auto_spell(text):
    corrected_text = ' '.join(spell(word) for word in text.split())
    return corrected_text

def preprocessing(data, return_array = False):
    # todo: better handling of na
    data = data.fillna('')
    for column in data.columns:
        print(column)
        # remove the digits and puntuation
        data[column] = data[column].str.replace('\d+', '')
        # convert to lowercase
        data[column] = data[column].str.replace('\W+', ' ')
        # replace continuous white spaces by a single one
        data[column] = data[column].str.replace('\s+', ' ')
        # words to lower
        data[column] = data[column].str.lower()
        # spell corrector 
        data[column] = data[column].apply(auto_spell)
        # lematize
        data[column] = data[column].apply(lemmatize_text)
        if not return_array:
            data[column] = data[column].str.join(' ')
    return data

In [64]:
train_df = preprocessing(train_df)
train_df.head()

sentence0
sentence1


,sentence0,sentence1
0,but other source close to the sale said vivend...,but other source close to the sale said vivend...
1,micron ha declared it first quarterly profit f...,micron s number also marked the first quarterl...
2,the fine are part of failed republican effort ...,perry said he back the senate s effort includi...
3,the american anglican council which represents...,the american anglican council which represents...
4,the tech loaded nasdaq composite rose point to...,the technology laced nasdaq composite index Ix...


In [65]:
test_df = preprocessing(test_df)
test_df.head()

sentence0
sentence1


,sentence0,sentence1
0,the problem likely will mean corrective change...,he said the problem need to be corrected befor...
1,the technology laced nasdaq composite index Ix...,the broad standard poor s index six inched up ...
2,it s a huge black eye said publisher arthur oc...,it s a huge black eye arthur sulzberger the ne...
3,sec chairman william Donaldson said there is a...,i think there s a building confidence that the...
4,vivendi share closed percent at euro in paris ...,in new york vivendi share were percent down at


### Lexical 

In [11]:
def lexical_simmilarity(df):
    guess = pd.DataFrame()
    for i in df.index:
        guess.loc[i,'labels'] = 1 - jaccard_distance(set(df.loc[i,'sentence0']), set(df.loc[i,'sentence1']))
    return guess

guess_lex = lexical_simmilarity(train_df)
guess_lex.head()

,labels
0,0.533333
1,0.388889
2,0.333333
3,0.607143
4,0.227273


## TfidVectorizer

In [66]:


stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]


def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))


tfv = TfidfVectorizer(max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')
# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(train_df['sentence1']) + list(train_df['sentence0']) )

def return_simil(a,b):
    simil = tfv.transform([a,b])
    return ((simil * simil.T).A)[0,1]

def calculate_all_sims(df):
    results = []
    for i in df.values:
        results.append(return_simil(i[0], i[1]))
    return results


all_sims = calculate_all_sims(train_df)

print('train pearson: ', pearsonr(all_sims, train_gs['labels'])[0])

test_sims = calculate_all_sims(test_df)
print('test pearson:', pearsonr(test_sims, test_gs['labels'])[0])

train pearson:  0.5002897027790068
test pearson: 0.5438793732392494


## Neural Networks

In [70]:
def test_model(model,xtrain,xtest):
    train_predicted =  model.predict(xtrain)
    test_predicted =   model.predict(xtest)
    print('train pearson: ', pearsonr(train_predicted, train_gs['labels'])[0])
    print('test pearson: ', pearsonr(test_predicted, test_gs['labels'])[0])

def train_and_test_model(model, train,test):
    model.fit(train,train_gs)
    test_model(model,train,test)

In [71]:
merged_sentences = train_df['sentence0'] + train_df['sentence1']
merged_test = test_df['sentence0'] + test_df['sentence1']

vectorizer = TfidfVectorizer(max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')
merged_train = vectorizer.fit_transform(merged_sentences)
merged_test = vectorizer.transform(merged_test)

In [72]:
model_nn = MLPRegressor(hidden_layer_sizes=(2,),validation_fraction=0.3, alpha=0.3,warm_start=False)
train_and_test_model(model_nn,merged_train,merged_test)

train pearson:  0.9596686530934233
test pearson:  0.4275063427960064


In [74]:
parameters = {'alpha': 10.0 ** -np.arange(0, 5), 'max_iter':[200],
              'hidden_layer_sizes':np.arange(1, 5),'solver': ['lbfgs','adam']}
nn_cv = GridSearchCV(MLPRegressor(), parameters, n_jobs=-1)
nn_cv.fit(merged_train,train_gs.values.ravel())
train_and_test_model(nn_cv,merged_train,merged_test)

train pearson:  0.9954971083991321
test pearson:  0.5296553224941408


## Random Forest

In [83]:
rfr = RandomForestRegressor(n_jobs=-1,n_estimators=500)
train_and_test_model(rfr,merged_train,merged_test)

train pearson:  0.9678412965317825
test pearson:  0.5672059537739355


## SVM

In [77]:
svr = SVR()
train_and_test_model(svr,merged_train,merged_test)

train pearson:  0.670542730484757
test pearson:  0.5375810030024217


In [78]:
words = re.compile(r"\w+",re.I)
stemmer = PorterStemmer()

def tokenize_sentences(df):
    sentence_0_tokenized = []
    sentence_1_tokenized = []

    for q in df.sentence0.tolist():
        sentence_0_tokenized.append([stemmer.stem(i.lower()) for i in words.findall(q) if i not in stopword])

    for q in df.sentence1.tolist():
        sentence_1_tokenized.append([stemmer.stem(i.lower()) for i in words.findall(q) if i not in stopword])

    df["sentence_0_tok"] = sentence_0_tokenized
    df["sentence_1_tok"] = sentence_1_tokenized
    
    return df

def train_dictionary(df):
    
    sentences_tokenized = df.sentence_0_tok.tolist() + df.sentence_1_tok.tolist()
    
    dictionary = corpora.Dictionary(sentences_tokenized)
    dictionary.filter_extremes(no_below=5, no_above=0.8)
    dictionary.compactify()
    
    return dictionary
    
tokenized_train = tokenize_sentences(train_df)
dictionary = train_dictionary(tokenized_train)
print ("No of words in the dictionary = %s" %len(dictionary))

tokenized_test = tokenize_sentences(test_df)

No of words in the dictionary = 1682


In [79]:
def get_vectors(df, dictionary):
    
    sentence0_vec = [dictionary.doc2bow(text) for text in df.sentence_0_tok.tolist()]
    sentence1_vec = [dictionary.doc2bow(text) for text in df.sentence_1_tok.tolist()]
    
    sentence0_csc = gensim.matutils.corpus2csc(sentence0_vec, num_terms=len(dictionary.token2id))
    sentence1_csc = gensim.matutils.corpus2csc(sentence1_vec, num_terms=len(dictionary.token2id))
    
    return sentence0_csc.transpose(),sentence1_csc.transpose()


q1_csc, q2_csc = get_vectors(tokenized_train, dictionary)
q1_csc_test, q2_csc_test = get_vectors(tokenized_test, dictionary)

print (q1_csc.shape)
print (q2_csc.shape)

(2234, 1682)
(2234, 1682)


In [80]:
train_bog = np.concatenate((q1_csc.todense(), q2_csc.todense()), axis=1)
test_bog = np.concatenate((q1_csc_test.todense(), q2_csc_test.todense()), axis=1)

In [84]:
train_and_test_model(rfr,train_bog,test_bog)

train pearson:  0.9660942421266484
test pearson:  0.591940433299118


## Try using all the distances

In [29]:
minkowski_dis = DistanceMetric.get_metric('minkowski')
mms_scale_man = MinMaxScaler()
mms_scale_euc = MinMaxScaler()
mms_scale_mink = MinMaxScaler()

def get_similarity_values(q1_csc, q2_csc):
    cosine_sim = []
    manhattan_dis = []
    eucledian_dis = []
    jaccard_dis = []
    minkowsk_dis = []
    
    for i,j in zip(q1_csc, q2_csc):
        sim = cs(i,j)
        cosine_sim.append(sim[0][0])
        sim = md(i,j)
        manhattan_dis.append(sim[0][0])
        sim = ed(i,j)
        eucledian_dis.append(sim[0][0])
        i_ = i.toarray()
        j_ = j.toarray()
        try:
            sim = jsc(i_,j_)
            jaccard_dis.append(sim)
        except:
            jaccard_dis.append(0)
            
        sim = minkowski_dis.pairwise(i_,j_)
        minkowsk_dis.append(sim[0][0])
    
    return cosine_sim, manhattan_dis, eucledian_dis, jaccard_dis, minkowsk_dis    


# cosine_sim = get_cosine_similarity(q1_csc, q2_csc)
cosine_sim, manhattan_dis, eucledian_dis, jaccard_dis, minkowsk_dis = get_similarity_values(q1_csc, q2_csc)

In [37]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

X_train = pd.DataFrame({"cos" : cosine_sim, "man" : manhattan_dis, "euc" : eucledian_dis, "jac" : jaccard_dis, "min" : minkowsk_dis})

X_test = pd.DataFrame({"cos" : y_pred_cos, "man" : y_pred_man, "euc" : y_pred_euc, "jac" : y_pred_jac, "min" : y_pred_mink})

rfr = RandomForestRegressor()
rfr.fit(X_train,train_gs.values.ravel())

svr = SVR()
svr.fit(X_train,train_gs.values.ravel())

y_rfr_predicted = rfr.predict(X_test)
y_svr_predicted = svr.predict(X_test)

print('test pearson: ', pearsonr(y_rfr_predicted, test_gs['labels'])[0])
print('test pearson: ', pearsonr(y_svr_predicted, test_gs['labels'])[0])


test pearson:  0.5243535486642252
test pearson:  0.5157266341490466


In [32]:
from sklearn.metrics import log_loss

def calculate_logloss(y_true, y_pred):
    loss_cal = log_loss(y_true, y_pred)
    return loss_cal

y_pred_cos, y_pred_man, y_pred_euc, y_pred_jac, y_pred_mink = get_similarity_values(q1_csc_test, q2_csc_test)




In [33]:
predictions = [y_pred_cos, y_pred_man, y_pred_euc, y_pred_jac, y_pred_mink]
for test_predicted in predictions:
    print('test pearson: ', pearsonr(test_predicted, test_gs['labels'])[0])

test pearson:  0.4611377260796854
test pearson:  -0.3223994960896658
test pearson:  -0.35691149798544414
test pearson:  0.44117147516557353
test pearson:  -0.35691149798544414
